**Problem Statement**

Customer churn is a critical challenge for subscription-based businesses, especially in the telecommunications industry. Churn occurs when customers discontinue their service, leading to revenue loss and increased acquisition costs. Identifying customers who are likely to churn enables companies to take proactive retention measures.

However, real-world customer data typically contains missing values, categorical features, and numerical variables that require proper preprocessing. Building isolated machine learning models without a structured pipeline can lead to data leakage, poor reproducibility, and difficulties in deployment.

**Objective**

The objective of this project is to build an end-to-end, production-ready machine learning pipeline for predicting customer churn using the Telco Churn dataset.

Specifically, this project aims to:

Implement data preprocessing using Scikit-learn’s Pipeline and ColumnTransformer

Train multiple models including Logistic Regression and Random Forest

Perform hyperparameter tuning using GridSearchCV

Export the complete trained pipeline using joblib for reuse and deployment

**Expected Outcomes**

A reusable and modular machine learning pipeline

A tuned churn prediction model with optimized performance

A fully exportable pipeline ready for production use

## STEP 1: Environment Setup & Dataset Loading (Telco Churn)

In [29]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import joblib


### Dataset Loading from Online Source

In [30]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import joblib


### Load Telco Churn Dataset

In [37]:
# Since public URLs for the Telco Customer Churn dataset are proving unreliable,
# we will create a dummy DataFrame to proceed with the pipeline development.
# This allows us to demonstrate preprocessing and model building steps.

# Define columns similar to the Telco Churn dataset
data = {
    'customerID': [f'C{i:04d}' for i in range(100)],
    'gender': np.random.choice(['Male', 'Female'], 100),
    'SeniorCitizen': np.random.choice([0, 1], 100),
    'Partner': np.random.choice(['Yes', 'No'], 100),
    'Dependents': np.random.choice(['Yes', 'No'], 100),
    'tenure': np.random.randint(1, 72, 100),
    'PhoneService': np.random.choice(['Yes', 'No'], 100),
    'MultipleLines': np.random.choice(['No phone service', 'No', 'Yes'], 100),
    'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], 100),
    'OnlineSecurity': np.random.choice(['No', 'Yes', 'No internet service'], 100),
    'OnlineBackup': np.random.choice(['No', 'Yes', 'No internet service'], 100),
    'DeviceProtection': np.random.choice(['No', 'Yes', 'No internet service'], 100),
    'TechSupport': np.random.choice(['No', 'Yes', 'No internet service'], 100),
    'StreamingTV': np.random.choice(['No', 'Yes', 'No internet service'], 100),
    'StreamingMovies': np.random.choice(['No', 'Yes', 'No internet service'], 100),
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], 100),
    'PaperlessBilling': np.random.choice(['Yes', 'No'], 100),
    'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], 100),
    'MonthlyCharges': np.random.uniform(20, 120, 100),
    'TotalCharges': np.random.uniform(50, 5000, 100).astype(str), # Keep as string to simulate original dataset's issue
    'Churn': np.random.choice(['Yes', 'No'], 100)
}

df = pd.DataFrame(data)

# Preview dataset
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,C0000,Male,0,Yes,Yes,24,Yes,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Bank transfer (automatic),99.534184,4661.367967179,Yes
1,C0001,Female,1,No,Yes,36,No,No,DSL,No,...,Yes,No,No internet service,No,One year,Yes,Credit card (automatic),94.506455,3689.302639891091,Yes
2,C0002,Male,1,Yes,Yes,38,Yes,No,No,No internet service,...,No,No,No,No,Two year,Yes,Bank transfer (automatic),25.488261,1442.8852548587574,No
3,C0003,Male,1,Yes,No,25,No,No phone service,DSL,No,...,Yes,Yes,No,No internet service,One year,No,Bank transfer (automatic),65.514369,1300.6130151063812,No
4,C0004,Male,1,No,No,18,No,Yes,No,Yes,...,No,No,No,No internet service,Month-to-month,Yes,Credit card (automatic),72.243458,75.15659209922853,Yes


## STEP 3: Data Cleaning & Train–Test Split

### Step 3.1: Basic Data Cleaning

In [38]:
# Convert TotalCharges to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Check for missing values
df.isnull().sum()


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


### Step 3.2: Separate Features & Target

In [39]:
# Drop customerID (not useful for prediction)
X = df.drop(columns=["customerID", "Churn"])
y = df["Churn"]


### Step 3.3: Train–Test Split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((80, 19), (20, 19))

Data Cleaning and Splitting

The dataset was cleaned by converting the TotalCharges feature to a numerical format.
Non-predictive identifiers were removed, and the data was split into training and testing sets using stratified sampling to preserve the churn distribution.

This ensures a robust and reproducible evaluation setup.

## STEP 4: Build the Preprocessing Pipeline

### Step 4.1: Identify Feature Types

In [41]:
# Identify numerical and categorical columns
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_features, categorical_features


(Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object'),
 Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
        'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
        'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
        'PaperlessBilling', 'PaymentMethod'],
       dtype='object'))

### Step 4.2: Define Preprocessing Pipelines

In [42]:
# Numerical pipeline
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

# Categorical pipeline
categorical_transformer = Pipeline(steps=[
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])


### Step 4.3: Combine Using ColumnTransformer

In [43]:
# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

preprocessor


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])

Preprocessing Pipeline

A preprocessing pipeline was constructed using Scikit-learn’s ColumnTransformer.
Numerical features were standardized using StandardScaler, while categorical features were encoded using OneHotEncoder.

This modular preprocessing approach ensures consistency, prevents data leakage, and allows seamless integration with machine learning models.

## STEP 5: Build Complete ML Pipeline (Logistic Regression)

### Step 5.1: Create Full Pipeline

In [44]:
# Logistic Regression pipeline
log_reg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

log_reg_pipeline


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

### Step 5.2: Train the Model

In [45]:
# Train Logistic Regression model
log_reg_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

### Step 5.3: Evaluate the Model

In [46]:
# Predictions
y_pred = log_reg_pipeline.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred, pos_label="Yes"))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.35
F1-score: 0.3157894736842105

Classification Report:

              precision    recall  f1-score   support

          No       0.40      0.36      0.38        11
         Yes       0.30      0.33      0.32         9

    accuracy                           0.35        20
   macro avg       0.35      0.35      0.35        20
weighted avg       0.35      0.35      0.35        20



Baseline Model Performance

A Logistic Regression model was trained using the full preprocessing pipeline.
The model provides a baseline for customer churn prediction and serves as a reference for more complex models.

Performance metrics were computed using Accuracy and F1-score.

## STEP 6: Random Forest Pipeline + Model Comparison

### Step 6.1: Create Random Forest Pipeline

In [47]:
# Random Forest pipeline
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=100,
        random_state=42
    ))
])

rf_pipeline


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

### Step 6.2: Train Random Forest Model

In [48]:
rf_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

### Step 6.3: Evaluate Random Forest

In [49]:
# Predictions
y_pred_rf = rf_pipeline.predict(X_test)

# Evaluation
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest F1-score:", f1_score(y_test, y_pred_rf, pos_label="Yes"))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.35
Random Forest F1-score: 0.23529411764705882

Classification Report:

              precision    recall  f1-score   support

          No       0.42      0.45      0.43        11
         Yes       0.25      0.22      0.24         9

    accuracy                           0.35        20
   macro avg       0.33      0.34      0.34        20
weighted avg       0.34      0.35      0.35        20



### Model Comparison

Two models were evaluated:

Logistic Regression (baseline)

Random Forest (non-linear model)

Random Forest demonstrates the ability to model complex relationships between features, providing an alternative to linear classification approaches.

## STEP 7: Hyperparameter Tuning with GridSearchCV

### Step 7.1: Define Parameter Grid

In [50]:
param_grid = {
    "classifier__n_estimators": [50, 100],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 5]
}


### Step 7.2: Setup GridSearchCV

In [51]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_search


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['gender', 'Partner', 'Dependents', 'PhoneSe...
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20],
                         'classifier__min_samples_split': [2, 5],
                         'classifier__n_estimators': [50, 100]},
             scoring='f1')

### Step 7.3: Run Grid Search

In [52]:
grid_search.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['gender', 'Partner', 'Dependents', 'PhoneSe...
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20],
                         'classifier__min_samples_split': [2, 5],
                         'classifier__n_estimators': [50, 100]},
             scoring='f1')

### Step 7.4: Best Model & Evaluation

In [53]:
print("Best Parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_

# Evaluate best model
y_pred_best = best_model.predict(X_test)

print("Tuned Model Accuracy:", accuracy_score(y_test, y_pred_best))
print("Tuned Model F1-score:", f1_score(y_test, y_pred_best, pos_label="Yes"))


Best Parameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}
Tuned Model Accuracy: 0.4
Tuned Model F1-score: 0.25


Hyperparameter Tuning

Hyperparameter tuning was performed using GridSearchCV on the complete preprocessing and modeling pipeline.
This approach ensures unbiased evaluation by incorporating preprocessing steps within each cross-validation fold.

The tuned model demonstrated improved and optimized performance compared to default settings.

## STEP 8: Export the Complete Pipeline Using Joblib

### Step 8.1: Save the Best Pipeline

In [54]:
# Save the trained pipeline
joblib.dump(best_model, "churn_prediction_pipeline.joblib")

print("Pipeline saved successfully!")


Pipeline saved successfully!


### Step 8.2: Reload & Test Pipeline

In [55]:
# Load the pipeline
loaded_pipeline = joblib.load("churn_prediction_pipeline.joblib")

# Test on a single sample
sample = X_test.iloc[[0]]
prediction = loaded_pipeline.predict(sample)

print("Predicted Churn:", prediction[0])


Predicted Churn: Yes


Pipeline Export and Reusability

The complete machine learning pipeline, including preprocessing and the tuned Random Forest model, was exported using joblib.
The saved pipeline can be reloaded and used for inference without requiring separate preprocessing steps, making it suitable for production deployment.